# Checkpointing in Workflows

During the execution of a Workflow, `Checkpoint` objects are automatically stored within the `Context` class. By default, `Checkpoints` are automatically created at the completion of a step. At such instants in the Workflow execution, we checkpoint the name of the last completed step, it's input `Event`, it's output `Event`, as well as a snapshot of the `Context` itself. We also store a `Checkpoint` before the emission of the `StartEvent`, but note that running from these special checkpoints are essentially equivalent to the typical execution of a `Workflow` -- we add it here purely for convenience.

In this notebook, we demonstrate how to make use of such checkpoints. Specifically, we'll show how to obtain the list of all of the stored checkpoints, filter this list, and even how to re-run a `Workflow` from any one them.

## Defining the Workflow

In [ ]:
import os

api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
from llama_index.core.workflow import (
    Workflow,
    step,
    StartEvent,
    StopEvent,
    Event,
    Context,
)
from llama_index.llms.openai import OpenAI


class JokeEvent(Event):
    joke: str


class JokeFlow(Workflow):
    llm = OpenAI(api_key=api_key)

    @step
    async def generate_joke(self, ev: StartEvent) -> JokeEvent:
        topic = ev.topic

        prompt = f"Write your best joke about {topic}."
        response = await self.llm.acomplete(prompt)
        return JokeEvent(joke=str(response))

    @step
    async def critique_joke(self, ev: JokeEvent) -> StopEvent:
        joke = ev.joke

        prompt = f"Give a thorough analysis and critique of the following joke: {joke}"
        response = await self.llm.acomplete(prompt)
        return StopEvent(result=str(response))

### Running Without Automatic Checkpointing

By default, automatic checkpointing is disabled. However, this can be enabled for any run via the `store_checkpoints` parameter, which we will show later in this notebook.

In [ ]:
# instantiate Jokeflow
workflow = JokeFlow()
# ctx = Context(workflow=workflow)

handler = workflow.run(
    topic="chemistry",
    # ctx=ctx,
    store_checkpoints=False,
)
await handler

'Analysis:\nThis joke plays on the double meaning of the word "rates," which can refer to both the cost of something (day rates) and the chemical compound nitrate. The humor comes from the unexpected twist in the punchline, where the listener is led to believe the joke is about the cost of nitrates but then it turns out to be a play on words.\n\nCritique:\nOverall, this joke is clever and plays on the knowledge of chemistry and wordplay. It requires a certain level of understanding of both concepts to fully appreciate the humor. However, some may find the joke to be a bit niche and not easily accessible to a general audience. Additionally, the punchline may not be as strong or laugh-out-loud funny as some other jokes, as it relies more on wordplay than a traditional setup and punchline structure. Overall, while the joke is clever and well-crafted, it may not appeal to everyone due to its niche subject matter.'

In [ ]:
len(workflow.checkpoints)

0

As we can see the number of stored checkpoints in our `Context` is 0.

### Running the Workflow With Checkpointing Enabled

In [ ]:
# run the workflow again, but this time with checkpointing enabled
handler = workflow.run(topic="math", store_checkpoints=True)
await handler

'Analysis:\nThis joke plays on the mathematical concept of the equal sign, which is used to show that two quantities are the same. The humor comes from personifying the equal sign and attributing human emotions and characteristics to it. The punchline relies on a play on words, as "less than" and "greater than" are mathematical symbols used to show inequality.\n\nCritique:\n- Clever wordplay: The joke cleverly uses mathematical symbols and concepts to create humor. The play on words with "less than" and "greater than" adds an extra layer of wit to the joke.\n- Relatable: The joke is relatable to anyone familiar with basic math, making it accessible to a wide audience.\n- Short and sweet: The joke is concise and to the point, making it easy to understand and appreciate quickly.\n- Lack of depth: While the joke is clever, it may be seen as somewhat shallow or simplistic. It relies on a basic pun and does not have a deeper underlying message or social commentary.\n- Predictable: The punch

### Listing the saved Checkpoints

There are 3 stored checkpoints after this first run. The first of these checkpoints are created just before the emission of the `StartEvent`. This can be thought of as a "startup" step being completed which has a null input `Event` and whose output event is the `StartEvent`. The remaining two of these checkpoints are made at the completion of each of the two steps in this Workflow. 

In [ ]:
workflow.checkpoints

{'a77e788e-bcd4-42cf-9981-2325f2e1fd4e': [Checkpoint(id_='5e466c0e-4158-4d46-ac22-ef9e616cfc5a', last_completed_step=None, input_event=None, output_event=StartEvent(run_id='a77e788e-bcd4-42cf-9981-2325f2e1fd4e'), ctx_state={'globals': {}, 'streaming_queue': '[]', 'queues': {'_done': '[]', 'critique_joke': '[]', 'generate_joke': '[]'}, 'stepwise': False, 'events_buffer': {}, 'accepted_events': [('generate_joke', 'StartEvent'), ('critique_joke', 'JokeEvent')], 'is_running': False}),
  Checkpoint(id_='49777d08-a638-4a5a-ba12-9bc1613f7fba', last_completed_step='generate_joke', input_event=StartEvent(run_id='a77e788e-bcd4-42cf-9981-2325f2e1fd4e'), output_event=JokeEvent(run_id='a77e788e-bcd4-42cf-9981-2325f2e1fd4e', joke="Why was the equal sign so humble?\n\nBecause he knew he wasn't less than or greater than anyone else."), ctx_state={'globals': {}, 'streaming_queue': '[]', 'queues': {'_done': '[]', 'critique_joke': '[]', 'generate_joke': '[]'}, 'stepwise': False, 'events_buffer': {}, 'acc

In [ ]:
for run_id, ckpts in workflow.checkpoints.items():
    print(f"Run: {run_id} has {len(ckpts)} stored checkpoints")

Run: a77e788e-bcd4-42cf-9981-2325f2e1fd4e has 3 stored checkpoints


### Filtering the Checkpoints

Before showcasing how to filter through checkpoints, let's first make things a bit more interesting by executing the workflow a few more times. Here, we'll run the workflow for two more additional topics. Since each entire run will result in 3 stored checkpoints each, we should have a total of 9 checkpoints after executing the below cell. 

In [ ]:
additional_topics = ["biology", "history"]

for topic in additional_topics:
    handler = workflow.run(topic=topic, store_checkpoints=True)
    await handler

In [ ]:
for run_id, ckpts in workflow.checkpoints.items():
    print(f"Run: {run_id} has {len(ckpts)} stored checkpoints")

Run: a77e788e-bcd4-42cf-9981-2325f2e1fd4e has 3 stored checkpoints
Run: fa60dd7c-70ea-46f4-a98b-8a9f9a93e2a2 has 3 stored checkpoints
Run: 38627bfd-64ac-43d0-9dc6-030b4e1520e6 has 3 stored checkpoints


In [ ]:
# Filter by the name of last completed step
checkpoints_right_after_generate_joke_step = workflow.filter_checkpoints(
    last_completed_step="generate_joke",
    run_id=list(workflow.checkpoints.keys())[1],
)

# checkpoint ids
[ckpt for ckpt in checkpoints_right_after_generate_joke_step]

[Checkpoint(id_='b2c3bc8c-9cde-4be2-84e8-b33b48bdbc56', last_completed_step='generate_joke', input_event=StartEvent(run_id='fa60dd7c-70ea-46f4-a98b-8a9f9a93e2a2'), output_event=JokeEvent(run_id='fa60dd7c-70ea-46f4-a98b-8a9f9a93e2a2', joke="Why did the biologist break up with the mathematician?\n\nBecause they couldn't find a common denominator!"), ctx_state={'globals': {}, 'streaming_queue': '[]', 'queues': {'_done': '[]', 'critique_joke': '[]', 'generate_joke': '[]'}, 'stepwise': False, 'events_buffer': {}, 'accepted_events': [('generate_joke', 'StartEvent'), ('critique_joke', 'JokeEvent')], 'is_running': True})]

In [ ]:
# Filter by output event StopEvent
checkpoints_that_emit_stop_event = workflow.filter_checkpoints(
    output_event_type=StopEvent
)

# checkpoint ids
[ckpt for ckpt in checkpoints_that_emit_stop_event]

[Checkpoint(id_='b1ab3216-70e2-4810-ac55-4c524d5ea3c1', last_completed_step='critique_joke', input_event=JokeEvent(run_id='a77e788e-bcd4-42cf-9981-2325f2e1fd4e', joke="Why was the equal sign so humble?\n\nBecause he knew he wasn't less than or greater than anyone else."), output_event=StopEvent(run_id='a77e788e-bcd4-42cf-9981-2325f2e1fd4e', result='Analysis:\nThis joke plays on the mathematical concept of the equal sign, which is used to show that two quantities are the same. The humor comes from personifying the equal sign and attributing human emotions and characteristics to it. The punchline relies on a play on words, as "less than" and "greater than" are mathematical symbols used to show inequality.\n\nCritique:\n- Clever wordplay: The joke cleverly uses mathematical symbols and concepts to create humor. The play on words with "less than" and "greater than" adds an extra layer of wit to the joke.\n- Relatable: The joke is relatable to anyone familiar with basic math, making it acce

### Re-Run Workflow from a specific checkpoint

In [ ]:
ckpt = checkpoints_right_after_generate_joke_step[0]
handler = workflow.run_from(checkpoint=ckpt, store_checkpoints=True)
await handler

'This joke plays on the common phrase "finding a common denominator," which is a mathematical term used when adding or subtracting fractions. In this context, the joke implies that the biologist and mathematician could not find a common ground or understanding between their two fields of study, leading to their breakup.\n\nOverall, the joke is clever and humorous in its use of wordplay and pun. It effectively combines elements of biology and mathematics to create a light-hearted and relatable scenario. The punchline is unexpected and plays on the idea of compatibility in relationships, adding an extra layer of humor.\n\nHowever, some may argue that the joke perpetuates stereotypes about the differences between scientists and mathematicians, suggesting that they are incompatible due to the nature of their respective fields. This could be seen as reductive and oversimplifying the complexities of individuals and their relationships.\n\nIn conclusion, while the joke is witty and entertaini

In [ ]:
for run_id, ckpts in workflow.checkpoints.items():
    print(f"Run: {run_id} has {len(ckpts)} stored checkpoints")

Run: a77e788e-bcd4-42cf-9981-2325f2e1fd4e has 3 stored checkpoints
Run: fa60dd7c-70ea-46f4-a98b-8a9f9a93e2a2 has 3 stored checkpoints
Run: 38627bfd-64ac-43d0-9dc6-030b4e1520e6 has 3 stored checkpoints
Run: ee135931-dc8c-4939-a09b-8ce5d7c536d6 has 1 stored checkpoints


Since we've executed from the checkpoint that represents the end of "generate_joke" step, there is only one additional checkpoint that gets stored i.e., the one that is stored after the completion of the next step, namely "critique_joke".